In [ ]:
import os 
import matplotlib.pyplot as plt
import numpy as np
import cv2
import re
import PIL
from IPython.display import Image, display
from scipy import spatial

In [ ]:
imgs = os.listdir('ImageNet_Workshop/Grad_CAM_output')
print(imgs)

In [ ]:
for img in imgs:
    # create figure
    fig = plt.figure(figsize=(25, 25))

    # setting values to rows and column variables
    rows = 1
    columns = 3
    
    gen_img = None
    perturb_img = None
    path_gen_img = "ImageNet_Workshop/generated_inputs/gen_img/"
    path_perturb = "ImageNet_Workshop/generated_inputs/perturb/"
    
    
    filenames = next(os.walk(path_gen_img), (None, None, []))[2] 
    for file in filenames:
        _img_path = file.split('_')
        _origin_img_id = _img_path[0]  # 0000000xxx
    
        if(_origin_img_id == img):
            gen_img = file
        
                 
    filenames = next(os.walk(path_perturb), (None, None, []))[2] 
    for file in filenames:
        _img_path = file.split('_')
        _origin_img_id = _img_path[0]  # 0000000xxx
        if(_origin_img_id == img):
            perturb_img = file
    """      
    filenames = next(os.walk(path_gen_img), (None, None, []))[2] 
    for file in filenames:
        m = re.search('(?<=\-)(.*?)(?=\-)', file)
        if m[1] == img:
            gen_img = file
            
    filenames = next(os.walk(path_perturb), (None, None, []))[2] 
    for file in filenames:
        m = re.search('(?<=\-)(.*?)(?=\-)', file)
        if m[1] == img:
            perturb_img = file"""
    
    # reading images
    Image1 = cv2.imread('ImageNet_Workshop/seeds_20/' + 'ILSVRC2012_test_' + img + '.JPEG')
    Image2 = cv2.imread(path_gen_img + gen_img)
    Image3 = cv2.imread(path_perturb + perturb_img)
    
    RGB_img1 = cv2.cvtColor(Image1, cv2.COLOR_BGR2RGB)
    RGB_img2 = cv2.cvtColor(Image2, cv2.COLOR_BGR2RGB)
    RGB_img3 = cv2.cvtColor(Image3, cv2.COLOR_BGR2RGB)

    # Adds a subplot at the 1st position
    fig.add_subplot(rows, columns, 1)

    # showing image
    plt.imshow(RGB_img1)
    plt.axis('off')
    plt.title(img + " (Original)")

    # Adds a subplot at the 2nd position
    fig.add_subplot(rows, columns, 2)

    # showing image
    plt.imshow(RGB_img2)
    plt.axis('off')
    plt.title(img + " (DLFuzz Generated)")
    
    # Adds a subplot at the 3nd position
    fig.add_subplot(rows, columns, 3)

    # showing image
    plt.imshow(RGB_img3)
    plt.axis('off')
    plt.title(img + " (Perturbation)")
    
    cams = next(os.walk('ImageNet_Workshop/Grad_CAM_output/' + img), (None, None, []))[2] 
    cams_original = next(os.walk('ImageNet_Workshop/Grad_CAM_output_originals/ILSVRC2012_test_' + img), (None, None, []))[2] 
    lowest_sim = 100
    Lowest_cam1 = ""
    lowest_cam2 = ""
    Lowest_perturb = ""
    for i in range(0,26):
        
        # calculate similarity
        img1 = PIL.Image.open('ImageNet_Workshop/Grad_CAM_output_originals/ILSVRC2012_test_' + img + '/' + cams_original[i])
        img2 = PIL.Image.open('ImageNet_Workshop/Grad_CAM_output/' + img + '/' + cams[i])
        
        img1 = img1.resize((round(img2.size[0]), round(img2.size[1])))
        img2 = img2.resize((round(img2.size[0]), round(img2.size[1])))
        
        cam_array1 = np.array(img1)
        cam_array2 = np.array(img2)
        
        cam_array1 = cam_array1.flatten()
        cam_array2 = cam_array2.flatten()
        
        cam_array1 = cam_array1/255
        cam_array2 = cam_array2/255
        
        similarity = -1 * (spatial.distance.cosine(cam_array1, cam_array2) - 1)
       
        # print("Cosine Sim.: ", "{:.4f}".format(similarity))
        
        if similarity < lowest_sim:
            lowest_sim = similarity
            lowest_heatmap1 = cams_original[i]
            lowest_heatmap2 = cams[i]
            lowest_cam1 = cams_original[i-1] 
            lowest_cam2 = cams[i-1] 
            
    # create figure
    fig_cam = plt.figure(figsize=(25, 25))

    # setting values to rows and column variables
    rows_cam = 3
    columns_cam = 3
    counter = 1
    
    img1 = PIL.Image.open('ImageNet_Workshop/Grad_CAM_output_originals/ILSVRC2012_test_' + img + '/' + lowest_cam1)
    img2 = PIL.Image.open('ImageNet_Workshop/Grad_CAM_output/' + img + '/' + lowest_cam2)

    img1 = img1.resize((round(img2.size[0]), round(img2.size[1])))
    img2 = img2.resize((round(img2.size[0]), round(img2.size[1])))

    fig_cam.add_subplot(rows_cam, columns_cam, counter)
    counter = counter + 1
    plt.imshow(img1)
    plt.axis('off')
    plt.title(lowest_cam1)

    fig_cam.add_subplot(rows_cam, columns_cam, counter)
    counter = counter + 1
    plt.imshow(img2)
    plt.axis('off')
    plt.title(lowest_cam2)

    fig_cam.add_subplot(rows_cam, columns_cam, counter)
    counter = counter + 1
    plt.imshow(RGB_img3)
    plt.axis('off')
    plt.title("Cosine Sim.: " + "{:.4f}".format(lowest_sim))
    
    img1 = PIL.Image.open('ImageNet_Workshop/Grad_CAM_output_originals/ILSVRC2012_test_' + img + '/' + lowest_heatmap1)
    img2 = PIL.Image.open('ImageNet_Workshop/Grad_CAM_output/' + img + '/' + lowest_heatmap2)

    img1 = img1.resize((round(img2.size[0]), round(img2.size[1])))
    img2 = img2.resize((round(img2.size[0]), round(img2.size[1])))

    fig_cam.add_subplot(rows_cam, columns_cam, counter)
    counter = counter + 1
    plt.imshow(img1)
    plt.axis('off')
    plt.title(lowest_cam1)

    fig_cam.add_subplot(rows_cam, columns_cam, counter)
    counter = counter + 1
    plt.imshow(img2)
    plt.axis('off')
    plt.title(lowest_cam2)

    fig_cam.add_subplot(rows_cam, columns_cam, counter)
    counter = counter + 1
    plt.imshow(RGB_img3)
    plt.axis('off')
    plt.title("Cosine Sim.: " + "{:.4f}".format(lowest_sim))
    